## Step 1: Verify Kaggle GPU Environment

First, let's confirm we have 2× Tesla T4 GPUs available.

In [ ]:
# Verify we have 2× T4 GPUs
import subprocess
import os

print("="*70)
print("🔍 KAGGLE GPU ENVIRONMENT CHECK")
print("="*70)

# Check nvidia-smi
result = subprocess.run(["nvidia-smi", "-L"], capture_output=True, text=True)
gpu_lines = [l for l in result.stdout.strip().split("\n") if l.startswith("GPU")]
print(f"\n📊 Detected GPUs: {len(gpu_lines)}")
for line in gpu_lines:
    print(f"   {line}")

# Check CUDA version
print("\n📊 CUDA Version:")
!nvcc --version | grep release

# Check total VRAM
print("\n📊 VRAM Summary:")
!nvidia-smi --query-gpu=index,name,memory.total --format=csv

# Verify we have 2 GPUs
if len(gpu_lines) >= 2:
    print("\n✅ Multi-GPU environment confirmed! Ready for llcuda v2.2.0.")
else:
    print("\n⚠️ WARNING: Less than 2 GPUs detected!")
    print("   Enable 'GPU T4 x2' in Kaggle notebook settings.")

## Step 2: Install llcuda v2.2.0

Install from GitHub with pre-built binaries for Kaggle T4×2.

In [ ]:
%%time
# Install llcuda v2.2.0 from GitHub
print("📦 Installing llcuda v2.2.0...")
!pip install -q git+https://github.com/llcuda/llcuda.git@v2.2.0

# Verify installation
import llcuda
print(f"\n✅ llcuda {llcuda.__version__} installed!")

# Check llcuda status
from llcuda.status import get_llcuda_status
status = get_llcuda_status()
print(f"\n📊 llcuda Status:")
print(f"   Binary Path: {status.get('binary_path', 'Not found')}")
print(f"   CUDA: {status.get('cuda_available', False)}")
print(f"   GPUs: {status.get('gpu_count', 0)}")

## Step 3: Download a GGUF Model

We'll use Gemma 3n 4B in Q4_K_M quantization - perfect for Kaggle T4 GPUs.

In [ ]:
%%time
from huggingface_hub import hf_hub_download
import os

# Model selection - optimized for 15GB VRAM
MODEL_REPO = "unsloth/gemma-3-4b-it-GGUF"
MODEL_FILE = "gemma-3-4b-it-Q4_K_M.gguf"

print(f"📥 Downloading {MODEL_FILE}...")
print(f"   Repository: {MODEL_REPO}")
print(f"   Expected size: ~2.5GB")

# Download to Kaggle working directory
model_path = hf_hub_download(
    repo_id=MODEL_REPO,
    filename=MODEL_FILE,
    local_dir="/kaggle/working/models"
)

print(f"\n✅ Model downloaded: {model_path}")

# Show model size
size_gb = os.path.getsize(model_path) / (1024**3)
print(f"   Size: {size_gb:.2f} GB")

## Step 4: Start llama-server

Start the inference server on GPU 0 with optimal settings for T4.

In [ ]:
from llcuda.server import ServerManager
from llcuda.api.multigpu import kaggle_t4_dual_config
import time

# Get optimized configuration for Kaggle T4
config = kaggle_t4_dual_config()

print("🚀 Starting llama-server...")
print(f"   Model: {model_path}")
print(f"   GPU Layers: {config.n_gpu_layers}")
print(f"   Context Size: {config.context_size}")
print(f"   Tensor Split: {config.tensor_split}")

# Create server manager
server = ServerManager()

# Start server with configuration
server.start(
    model_path=model_path,
    host="127.0.0.1",
    port=8080,
    n_gpu_layers=config.n_gpu_layers,
    context_size=config.context_size,
    flash_attn=True,
    main_gpu=0  # Use GPU 0 for LLM
)

# Wait for server to be ready
print("\n⏳ Waiting for server to load model...")
if server.wait_until_ready(timeout=120):
    print("\n✅ llama-server is ready!")
    print(f"   API endpoint: http://127.0.0.1:8080")
else:
    print("\n❌ Server failed to start within timeout")

## Step 5: Run Your First Inference

Use the OpenAI-compatible API to chat with the model.

In [ ]:
from llcuda.api.client import LlamaCppClient

# Create client
client = LlamaCppClient(base_url="http://127.0.0.1:8080")

# Test simple completion
print("💬 Testing inference...\n")

response = client.chat_completion(
    messages=[
        {"role": "user", "content": "What is CUDA? Explain in 2 sentences."}
    ],
    max_tokens=100,
    temperature=0.7
)

print("📝 Response:")
print(response.choices[0].message.content)

print(f"\n📊 Stats:")
print(f"   Tokens generated: {response.usage.completion_tokens}")
print(f"   Total tokens: {response.usage.total_tokens}")

## Step 6: Streaming Response Example

Stream responses for real-time output.

In [ ]:
# Streaming example
print("💬 Streaming response...\n")

for chunk in client.chat_completion_stream(
    messages=[
        {"role": "user", "content": "Write a Python function to calculate factorial."}
    ],
    max_tokens=200,
    temperature=0.3
):
    if chunk.choices and chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

print("\n\n✅ Streaming complete!")

## Step 7: Check GPU Memory Usage

Monitor VRAM usage to understand resource consumption.

In [ ]:
# Check GPU memory usage
print("📊 GPU Memory Usage:")
print("="*60)
!nvidia-smi --query-gpu=index,name,memory.used,memory.total,utilization.gpu --format=csv

print("\n💡 Note:")
print("   GPU 0: llama-server (LLM inference)")
print("   GPU 1: Available for RAPIDS/Graphistry")

## Step 8: Cleanup

Stop the server when done.

In [ ]:
# Stop the server
print("🛑 Stopping llama-server...")
server.stop()
print("\n✅ Server stopped. Resources freed.")

# Verify GPU memory is released
print("\n📊 GPU Memory After Cleanup:")
!nvidia-smi --query-gpu=index,memory.used,memory.total --format=csv

## 🎉 Quick Start Complete!

You've successfully:
1. ✅ Verified Kaggle GPU environment
2. ✅ Installed llcuda v2.2.0
3. ✅ Downloaded a GGUF model
4. ✅ Started llama-server
5. ✅ Ran inference with chat completion
6. ✅ Used streaming responses

## Next Steps

Explore more tutorials:
- 📘 [02-llama-server-setup](02-llama-server-setup-llcuda-v2.2.0.ipynb) - Advanced server configuration
- 📘 [03-multi-gpu-inference](03-multi-gpu-inference-llcuda-v2.2.0.ipynb) - Dual T4 inference
- 📘 [04-gguf-quantization](04-gguf-quantization-llcuda-v2.2.0.ipynb) - Quantization guide
- 📘 [05-unsloth-integration](05-unsloth-integration-llcuda-v2.2.0.ipynb) - Unsloth training → llcuda
- 📘 [06-split-gpu-graphistry](06-split-gpu-graphistry-llcuda-v2.2.0.ipynb) - LLM + Graph visualization

---

**llcuda v2.2.0** | CUDA 12 Inference Backend for Unsloth